In [1]:
%%capture
!pip install causal-conv1d>=1.1.0 mamba-ssm pytorch-minimize psutil gputil

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import psutil
import GPUtil
from google.colab import drive
from mamba_ssm import Mamba
import time
from pathlib import Path

In [3]:
# This will prompt you to click on a link and generate an authorization code.
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/AI/CGD_research/light-har'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def compute_metrics_base(model, x_test, y_test, model_path):
    """
    Compute the accuracy of the PyTorch model.

    :param model: PyTorch model.
    :param x_test: Test dataset features (as a PyTorch Tensor).
    :param y_test: Test dataset labels (as a NumPy array).
    :param model_dir: Directory where the PyTorch model files are stored.
    :return: None
    """

    model.eval()
    with torch.no_grad():
        # Get the model's predictions
        outputs = model(x_test)
        _, predicted_labels = torch.max(outputs, 1)

        # Convert y_test to tensor if it's not already
        true_labels = torch.tensor(y_test) if not isinstance(y_test, torch.Tensor) else y_test
        true_labels = true_labels.squeeze()  # Remove unnecessary dimensions

    model_file = Path(model_path)

    # Size in bytes
    model_size_bytes = model_file.stat().st_size

    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

    # Compute accuracy
    accuracy = accuracy_score(true_labels.cpu().numpy(), predicted_labels.cpu().numpy())
    print(f'Accuracy on the test set: {accuracy:.2%}')

def measure_cpu_utilization_and_run(func, *args, **kwargs):
    """
    Measure CPU utilization while running a function.

    Parameters:
        func (function): The function to be executed.
        *args: Arguments to be passed to func.
        **kwargs: Keyword arguments to be passed to func.

    Returns:
        float: CPU utilization percentage during the execution of func.
        float: The elapsed time during the execution of func.
        any: The result of func execution.
    """

    # Measure CPU utilization before execution
    cpu_percent_before = psutil.cpu_percent(interval=None)

    # Record the start time
    start_time = time.time()

    # Execute the function and store its result
    result = func(*args, **kwargs)

    # Record the end time
    end_time = time.time()

    # Measure CPU utilization after execution
    cpu_percent_after = psutil.cpu_percent(interval=None)

    # Calculate elapsed time and average CPU utilization
    elapsed_time = end_time - start_time
    average_cpu_utilization = (cpu_percent_before + cpu_percent_after) / 2

    return average_cpu_utilization, elapsed_time, result

In [5]:
# Load Data
x = genfromtxt(project_path+'/data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv(project_path+'/data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [20]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(x_train_tensor.shape)

torch.Size([104856, 16, 3])


In [7]:
# Create a DataLoader
batch_size = 32
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Create a test Dataloader
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [8]:
class TimeSeriesMamba(nn.Module):
    def __init__(self, sequence_length, num_features, head_size, n_heads, ff_dim, n_trans_blocks, mlp_units, drop=0.0, mlp_drop=0.0):
        super(TimeSeriesMamba, self).__init__()

        # Encoder module
        self.encoders = nn.ModuleList([
            Mamba(
                d_model=num_features,
                d_state=16,
                d_conv=4,
                expand=2,
        ) for _ in range(n_trans_blocks)])

        # Global average pooling
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)

        # MLP layers
        mlp_layers = []
        current_dim = num_features
        for dim in mlp_units:
            mlp_layers.extend([
                nn.Linear(current_dim, dim),
                nn.ReLU(),
                nn.Dropout(mlp_drop) if mlp_drop > 0.0 else nn.Identity(),  # Apply dropout conditionally
            ])
            current_dim = dim

        self.mlp = nn.Sequential(*mlp_layers)
        self.final_layer = nn.Linear(mlp_units[-1], 6)

    def forward(self, input_data):
        # Encoder
        for encoder in self.encoders:
            encoded_data = encoder(input_data)

        # Global average pooling
        encoded_data = encoded_data.permute(0, 2, 1)
        global_avg_pooled = self.global_avg_pooling(encoded_data)
        flattened = torch.flatten(global_avg_pooled, 1)

        # MLP
        mlp_output = self.mlp(flattened)

        # Final layer
        output = self.final_layer(mlp_output)
        return output

# Model instantiation
model = TimeSeriesMamba(
    sequence_length=16,
    num_features=3,
    head_size=3,
    n_heads=1,
    ff_dim=64,
    n_trans_blocks=4,
    mlp_units=[128, 64],
    drop=0.1,
    mlp_drop=0.1
)

## Trained by Adam Optimizer

In [15]:
model_path = project_path+'/models/mamba_base.pt'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [21]:
model = model.to('cuda')
x_test_tensor = x_test_tensor.to('cuda')
y_test_tensor = y_test_tensor.to('cuda')

cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, model, x_test_tensor, y_test_tensor, model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')

Size of the model: 55.66 KB
Accuracy on the test set: 86.92%
CPU usage during inference: 58.65%
Inference time: 0.2846 seconds


## Trained by Conjugate Gradient Method

In [57]:
model_path = project_path+'/models/mamba_cg_base.pt'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [58]:
model = model.to('cuda')
x_test_tensor = x_test_tensor.to('cuda')
y_test_tensor = y_test_tensor.to('cuda')

cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, model, x_test_tensor, y_test_tensor, model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')

Size of the model: 55.79 KB
Accuracy on the test set: 78.53%
CPU usage during inference: 40.10%
Inference time: 0.0664 seconds
